<a href="https://colab.research.google.com/gist/GeraldKaeferle/cca57d4db4fa34993c0e0771f62810d8/text-summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from nltk import download
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.tag import pos_tag
from nltk.stem.snowball import SnowballStemmer
import datetime
from dateutil.parser import parse
import numpy as np
from numpy import dot
from numpy.linalg import norm
import networkx as nx
import requests
from urllib.parse import urlparse
import unicodedata
from bs4 import BeautifulSoup
from scipy import stats
from matplotlib import pyplot
import timeit


In [124]:
def cos_sim(a,b):
  return (a @ b.T) / (norm(a)*norm(b))

a = np.array([3, 45, 7, 2])
b = np.array([2, 54, 13, 15])

cos_sim(a,b)

0.9722842517123499

In [0]:
!pip install wikipedia
import wikipedia

In [2]:
download(['stopwords','punkt','averaged_perceptron_tagger','wordnet'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
def read_text_article(source, n_sentences=-1):
    article = {"error":False}
    if source.find("http")==0:
      res = requests.get(source)
      cookies = dict(res.cookies)
      if res.status_code == 200:
        filedata = ' '.join(res.content.decode('utf-8', errors='ignore').splitlines())
        #filedata = filedata.decode('utf-8')
        #if filedata.encoding == 'utf-8':
        filedata = unicodedata.normalize('NFKC',filedata)
      else:
        print("Fehler: {}".format(res.status_code))
    else:
      file = open(source, "r")
      filedata = file.readlines()

    article["text"] = sent_tokenize(filedata)
    sentences = []

    for sentence in article["text"]:
        #print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
        
    sentences.pop()
    if n_sentences != -1:
      sentences=sentences[0:n_sentences]

    article["sentences"] = sentences  
    return article

def read_online_article(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 5.1.1; SM-G928X Build/LMY47X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.83 Mobile Safari/537.36'}
    sess = requests.Session()
    res=sess.get(url, headers=headers)
    cookies = dict(res.cookies)
    res = sess.post(url, cookies=cookies)
    article = {"error":False}
    if res.status_code == 200:
      soup = BeautifulSoup(res.content.decode('utf-8',errors='ignore'), "html.parser")
      # extract content
      text=[]

      article["tags"]=[]
      for tag in soup.find_all("meta"):
        if tag.get("property", None) == "og:title":
          article["title"]   = tag.get("content", None)
        elif tag.get("property", None) == "article:published_time":
          article["date"]    = tag.get("content", None)
        elif tag.get("property", None) == "og:locale":
          article["locale"]   = tag.get("content", None)
        elif tag.get("property", None) == "og:type":
          article["type"]   = tag.get("content", None)
        elif tag.get("property", None) == "article:tag":
          article["tags"].append(tag.get("content", None))
        elif tag.get("property", None) == "og:description":
          article["description"]   = tag.get("content", None)
        elif tag.get("property", None) == "og:site_name":
          article["site_name"]   = tag.get("content", None)

      if "title" != article.values():
          article["title"]   = soup.find("title").get_text().strip()
 
      if urlparse(url).netloc   == 'www.electrive.net':
        article["text"] = soup.find("main",attrs={"id": "main"}).get_text()
      elif urlparse(url).netloc == 'help.orf.at':
        article = soup.find("div" , attrs={"class": "storyText", "role":"article"})
        title   = article.find("title")
        article = article.get_text()
      elif urlparse(url).netloc.find("orf.at") >-1:
        title   = soup.find("h1", attrs={"class": "story-lead-headline"}).get_text()
        article["text"] = soup.find("div", attrs={"class": "story-story"}).get_text()
      elif urlparse(url).netloc == 'www.solarserver.de':
        article["text"] = soup.find("div", attrs={"class": "postContent bodyCopy entry-content clearfix"}).get_text()
      #elif urlparse(url).netloc == 'www.eon.com':
       #article["text"] = soup.find("div", attrs={"class":"content-body"}).get_text()
       # article["text"] = soup.find("div", attrs={"class":"content-body", "id":"content", "role":"main"}).get_text()
      elif urlparse(url).netloc == 'www.derbrutkasten.com':
        article["text"] = soup.find("div", attrs={"id": "main-article"}).get_text()
      elif urlparse(url).netloc ==  'www.ots.at':
        article["text"] = soup.find("div", attrs={"class": "content"}).get_text()
      elif urlparse(url).netloc == 'futurezone.at':
        article["text"] = soup.find("div", attrs={"class": "article-paragraphs"}).get_text()
      elif urlparse(url).netloc.find('heise.de') >-1:
        article["text"] = soup.find("article").get_text()
      elif urlparse(url).netloc.find('golem.de') >-1:
        article["text"] = soup.find("article").get_text()
      elif urlparse(url).netloc == 'www.sonnewindwaerme.de':
        article["text"] = soup.find("div",attrs={"class":"text-wrapper"}).get_text()
      elif urlparse(url).netloc == 'industriemagazin.at':
        article["text"] = soup.find("div",attrs={"class":"text-wrapper"}).get_text()
      elif urlparse(url).netloc in ('www.speicherbranche.de','www.solarbranche.de','www.windbranche.de','www.energiefirmen.de','www.bioenergie-branche.de'):
        article["text"] = soup.find("div",attrs={"class":"newsdiv"}).get_text().strip()
      elif urlparse(url).netloc == 'www.iwr.de':
        article["text"] = soup.find("div",attrs={"id":"content"}).get_text()
      #elif urlparse(url).netloc == 'www.springerprofessional.de':
      #  article["text"] = soup.find("div", attrs={"class":"rich-text mtl"}).get_text()
      elif urlparse(url).netloc == 'www.alliander.de':
        article["text"] = soup.find("div", attrs={"class":"entry-content", "itemprop":"text"}).get_text()
      else:
        text  = soup.find_all("p")
        sents=[]
        for p in text:
          for sent in sent_tokenize(p.get_text()):
            sents.append(sent)
        article["text"] = " ".join(sents)

      text = sent_tokenize(article["text"].strip())
    else:
      print("Fehler: {}".format(res.status_code))
      article = {"error":True}
      return article

    sentences = []

    for sentence in text:
        sentence = " ".join(sentence.split())
        #print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
        
    sentences.pop()
    article["sentences"] = sentences
    #print(len(sentences))
    return article

def read_wikipedia_article(title):
    wiki = wikipedia.page(title)
    article = sent_tokenize(wiki.content)
    sentences = []

    for sentence in article:
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    #print(len(sentences))
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(source, top_n=5, url=False, wiki=False, lang='english', n_sentences=-1):

    summarize_text = []
    article={}

    # Step 1 - Read text anc split it
    if url == True:
      article = read_online_article(source)
    elif wiki == True:
      sentences =  read_wikipedia_article(source)
    else:
      article = read_text_article(source, n_sentences)
    
    if article == {"error":True}:
      return article
    
    sentences = article["sentences"]
    
    if "locale" in article.keys():
      if article["locale"][0:2]  =="de":
        lang = "german"
      elif article["loacle"][0:2]  =="en":
        lang = "english"

    stop_words = stopwords.words(lang)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    #print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize text
    article["summary"] = " ".join(summarize_text)
    #print("Summarize Text: \n", summary)
    return article
    
def preprocess(sent):
    sent = word_tokenize(sent)
    sent = pos_tag(sent)
    return sent

In [4]:
article = 'http://www.gutenberg.org/files/53713/53713-0.txt'
article = generate_summary( article, 3, lang='german', n_sentences=100)
if article["error"]==False:
  for k in ("title","date", "summary"):
    if k in article.keys():
      print(k)
      print(article[k])

summary
Mir wird heute noch ganz schwindelig, wenn ich daran denke, wie schön sie war. Ich hab' ganz genau gewußt, daß sie am Sonntag in die Schottenkirche geht, ich hab' gewußt, wann ich sie am Nachmittag in der Stadt treffe. Vor dreißig Jahren bin ich nämlich öfter stehengeblieben und hab' ihr nachgeschaut.


In [9]:
#url='https://orf.at/stories/3161835/'
#url='https://orf.at/stories/3162239/'
#url='https://help.orf.at/stories/3001436/'
#url='https://noe.orf.at/stories/3044405/'
#url="https://www.electrive.net/2020/04/16/new-mobility-enterprise-bringt-elektro-van-aus-china-nach-oesterreich/"
#url="http://www.solarserver.de/solar-magazin/nachrichten/aktuelles/2015/kw51/sonnen-gmbh-startet-auslieferung-des-intelligenten-batteriespeicher-systems-sonnenbatterie-in-den-usa.html"
url="https://www.eon.com/de/ueber-uns/presse/press-releases/2019/2019-07-19-eon-applies-for-digital-patent-for-full-customer-data-control.html"
#url="https://www.ots.at/presseaussendung/OTS_20200319_OTS0119/coronavirus-versorgungssicherheit-als-hoechstes-gut"
#url="https://www.ots.at/presseaussendung/OTS_20160606_OTS0028/pwc-studie-neue-technologiefaktoren-veraendern-finanzdienstleistungen?utm_source=2016-06-06&utm_medium=email&utm_content=html&utm_campaign=mailabodigest"
#url="https://futurezone.at/apps/google-startet-augmented-reality-plattform-arcore-10/400004972"
#url="https://www.derbrutkasten.com/wiener-stadtwerke-grape/"
#url="https://www.heise.de/newsticker/meldung/Einstein-behaelt-Recht-Stern-rast-in-Rosettenbahn-um-Schwarzes-Loch-4703533.html"
#url="https://www.golem.de/news/corsa-e-probe-gefahren-ein-kleiner-flitzer-soll-die-masse-elektrifizieren-2004-147893.html"
#Fehler!#url="https://www.sonnewindwaerme.de/panorama/ammoniak-nachhaltiger-energietraeger"
#url="https://industriemagazin.at/a/deutsche-behoerden-verhindern-millionenbetrug-mit-windkraft"
#url='https://www.speicherbranche.de/news/nachrichten/artikel-36680-tesvolt-steigt-in-die-grossproduktion-von-batteriespeichern-ein-'
#url="https://www.solarbranche.de/news/nachrichten/artikel-36694-encavis-schuettet-planmaessig-hoehere-dividende-aus"
#url="https://www.iwr.de/news.php?id=36657"
#url="https://www.derstandard.at/story/2000024337615/tesla-hausbatterie-kommt-nach-oesterreich"
#url="https://www.springerprofessional.de/energie/energiespeicher/energie-aus-der-nachbarschaft-speichern-und-nutzen/6595482"
#url="https://www.solarserver.de/solar-magazin/nachrichten/aktuelles/2018/kw06/schwedische-dorfbewohner-versorgen-sich-komplett-selbst-mit-energie.html"
#url='https://www.alliander.de/alliander-startet-ersten-flexibilitaetsmarkt-in-den-niederlanden/'
#url='https://www.alliander.de/stadtwerke-begreifen-die-digitalisierung-mehrheitlich-als-chance/'
#url='https://www.smart2zero.com/news/450kw-charging-station-goes-online/page/0/1'
#url='https://www.i-magazin.com/entspannung-gesucht/'
#url="https://www.faz.net/aktuell/wirtschaft/neue-mobilitaet/sharing-modell-daimler-plant-car2go-fuer-gewerbekunden-14893806.html?GEPC=s5"
article = generate_summary( url, 3, url=True, lang='german')
if article["error"]==False:
  for k in ("title","date", "summary","tags"):
    if k in article.keys():
      print(k)
      print(article[k])

#preprocess(summary)

IndexError: ignored

In [228]:
url="https://www.derstandard.at/story/2000024337615/tesla-hausbatterie-kommt-nach-oesterreich"
#url="https://www.springerprofessional.de/energie/energiespeicher/energie-aus-der-nachbarschaft-speichern-und-nutzen/6595482"
#url='https://www.eon.com/de/ueber-uns/presse/pressemitteilungen/2016/2016-04-05-eon-a-core-partner-in-project-to-promote-energy-transition.html'
#url='https://www.alliander.de/stadtwerke-begreifen-die-digitalisierung-mehrheitlich-als-chance/'
sess= requests.Session()
res=sess.get(url)
cookies=dict(res.cookies)
print(res.status_code)
#res=sess.post(url,cookies=cookies)
soup = BeautifulSoup(res.content, "html.parser")
#soup.get_text()
text  = soup.find_all("p")
sents=[]
for p in text:
  for sent in sent_tokenize(p.get_text()):
    sents.append(sent)
sents
#sent_tokenize(text)

#for line in text:
#  if line in ['\n','','.']:
#    continue
#  for token in sent_tokenize(line):
#    sents.append(token) 
#sents
#text = soup.find("div", attrs={"class":"content-body", "id":"content", "role":"main"}).get_text().strip()
#sent_tokenize(text)
#soup.find("title").get_text().strip()

200


['Die Powerwall speichert selbsterzeugten Strom, der Verbund führt die Technologie hierzulande ein',
 'Wien – Sie wird an die Wand gehängt und ist mit Kapazitäten von sieben kWh erhältlich, bei Stromausfällen dient sie als Backup und in Smart-Grids als Sparschwein: Mit diesem Konzept will der US-Elektroautopionier Tesla die Energieversorgung von Häusern revolutionieren.',
 'Nun kommt die Powerwall nach Österreich, kündigte der Verbund am Donnerstag an.',
 'Ich stimme der Verwendung von Cookies für die Zwecke der Webanalyse und digitaler Werbemaßnahmen zu.',
 'Auch wenn ich diese Website weiter nutze, gilt dies als Zustimmung.',
 'Meine Einwilligung kann ich hier widerrufen.',
 'Weitere Informationen finde ich in der Datenschutzerklärung.',
 'Es ist ein Fehler aufgetreten.',
 'Bitte versuchen Sie es später erneut.',
 'Sie haben Cookies deaktiviert.',
 'Sie müssen Cookies in Ihrem Browser aktivieren und die Seite neu laden, um zustimmen zu können.',
 'Abo ohne Daten-Zustimmung',
 'Mit ei

In [238]:
'\xc0msterdam'

'Àmsterdam'

In [0]:
stemmer= SnowballStemmer("german", ignore_stopwords=True)
pos=[]
for token in pos_tag(word_tokenize(article["text"])):
  if token[1] in ('NNP'): 
    pos.append(stemmer.stem(token[0]))
    #pos.append(token[0])
#pos
fdist = FreqDist(word.lower() for word in pos)
fdist.pformat(40)

"FreqDist({'der': 16, 'die': 12, 'und': 9, 'mitarbeit': 8, 'mitarbeiterinn': 6, 'zu': 5, 'netz': 4, 'von': 4, 'arbeit': 4, 'coronavirus': 3, 'kelag-konz': 3, 'ist': 3, 'kng-karnt': 3, 'gmbh': 3, 'im': 3, 'auf': 3, 'kontakt': 3, 'für': 3, 'sowi': 3, 'bis': 3, 'strom-': 2, 'erdgas-': 2, 'warmeversorg': 2, 'klagenfurt': 2, 'energie-': 2, 'netzleitstell': 2, 'kelag': 2, 'uhr': 2, 'das': 2, 'vor': 2, 'umgesetzt': 2, 'hygienemassnahm': 2, 'verantwort': 2, 'betrieb': 2, 'unter': 2, 'umsetzungsphas': 2, 'mit': 2, 'haus': 2, 'versorgungssich': 1, 'gut': 1, ...})"

In [0]:
#Wikipedia
wikipedia.set_lang("en")
summary = generate_summary( "Keke Rosberg",2, wiki=True, lang='english')
preprocess(summary)

In [0]:
wikipedia.search("Mensch", results=3)

In [0]:
wikipedia.summary("Mensch", sentences=5)

In [10]:
!pip install spacy_sentiws

  Created wheel for spacy-sentiws: filename=spacy_sentiws-0.0.6-cp36-none-any.whl size=3537 sha256=eda4f711f5a82d596eebd1d65487386f01d250811639ada8485648f5f70bfafe
  Stored in directory: /root/.cache/pip/wheels/cd/d3/36/cc97ed49b6a64788fdd0d0e28abad04accca98aea066a4e47d
Successfully built spacy-sentiws


In [0]:
import spacy
from spacy_sentiws import spaCySentiWS
from collections import Counter
from string import punctuation

In [47]:
!python -m spacy download de_core_news_sm
!python -m spacy download de_core_news_md
!python -m spacy download xx_ent_wiki_sm
!python -m spacy download en_core_web_lg
#!python -m spacy download de_core_news_md
!python -m spacy validate

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
     |████████████████████████████████| 224.6MB 1.2MB/s 
  Created wheel for de-core-news-md: filename=de_core_news_md-2.2.5-cp36-none-any.whl size=228399478 sha256=a393f55e0cdbc72118f909e60c60ac039794a87eef665b18e4e1d085b63c7794
  Stored in directory: /tmp/pip-ephem-wheel-cache-8pssaa62/wheels/41/60/41/81898870259d7c19fe8f9e46a537611c939f0c425eee2e1785
Successfully built de-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_md')
✔ Download and installation successful
You can now load the model via spacy.load('xx_ent_wiki_sm')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
✔ Loaded compatibility table

====================== Installed models (spaCy v2.2.4) ======================
ℹ spaCy installation: /usr/local/lib/python3.6/dist-packages/spacy

TYPE      NAME              MODEL  

In [0]:
!wget http://pcai056.informatik.uni-leipzig.de/downloads/etc/SentiWS/SentiWS_v2.0.zip
!unzip SentiWS_v2.0.zip

In [0]:
spacy.prefer_gpu()
import de_core_news_md
import en_core_web_lg
import xx_ent_wiki_sm
nlp_de = de_core_news_md.load()
nlp_en = en_core_web_lg.load()
nlp_xx = xx_ent_wiki_sm.load()
sentiws = spaCySentiWS(sentiws_path='/content')
nlp_de.add_pipe(sentiws)

In [0]:
def top_sentence(text, limit, lang):
    keyword = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
    if lang == 'de':   
      doc = nlp_de(text)
    elif lang == 'en':
      doc = nlp_en(text)
    else:
      doc = nlp_xx(text)
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            keyword.append(token.text)
    
    freq_word = Counter(keyword)
    max_freq = Counter(keyword).most_common(1)[0][1]
    for w in freq_word:
        freq_word[w] = (freq_word[w]/max_freq)
        
    sent_strength={}
    for sent in doc.sents:
        for word in sent:
            if word.text in freq_word.keys():
                if sent in sent_strength.keys():
                    sent_strength[sent]+=freq_word[word.text]
                else:
                    sent_strength[sent]=freq_word[word.text]
    
    summary = []
    
    sorted_x = sorted(sent_strength.items(), key=lambda kv: kv[1], reverse=True)
    
    counter = 0
    for i in range(len(sorted_x)):
        summary.append(str(sorted_x[i][0]))

        counter += 1
        if(counter >= limit):
            break
            
    return ' '.join(summary)

In [0]:
example_text = '''Yamaha is reminding people that musical equipment cases are for musical equipment — not people — two weeks after fugitive auto titan Carlos Ghosn reportedly was smuggled out of Japan in one. In a tweet over the weekend, the Japanese musical equipment company said it was not naming any names, but noted there had been many recent stories about people getting into musical equipment cases. Yamaha (YAMCY) warned people not to get into, or let others get into, its cases to avoid "unfortunate accidents." Multiple media outlets have reported that Ghosn managed to sneak through a Japanese airport to a private jet that whisked him out of the country by hiding in a large, black music equipment case with breathing holes drilled in the bottom. CNN Business has not independently confirmed those details of his escape. The former Nissan (NSANF) CEO had been out on bail awaiting trial in Japan on charges of financial wrongdoing before making his stunning escape to Lebanon at the end of December. Ghosn has referred to his departure as an effort to "escape injustice." In an interview with CNN\'s Richard Quest last week, Ghosn did not comment on the nature of his escape, saying he didn\'t want to endanger any of the people who aided in the operation. Ghosn did, however, respond to a question about what it felt like to ride through the airport in a packing case by first declining to comment but then adding: "Freedom, no matter the way it happens, is always sweet." In a press conference in Lebanon ahead of the CNN interview last Wednesday, Ghosn\'s first public appearance since fleeing Japan, Ghosn said he decided to leave the country because he believed he would not receive a fair trial, a claim Japanese authorities have disputed. Brands sometimes capitalize on their tangential relationship to big news in order to attract attention on social media. Yamaha is one of Japan\'s best known brands and Ghosn was one of Japan\'s top executives before being ousted from Nissan — a match made in social media heaven. Not surprisingly, Yamaha\'s post went viral on Twitter over the weekend.'''

In [0]:
example_text =article["text"]

In [168]:
doc = nlp(example_text)
sentiment=[]
for token in doc:
  #print('{}, {}, {}'.format(token.text, token._.sentiws, token.pos_))
  if token._.sentiws != None:
    sentiment.append(token._.sentiws)
np.mean(sentiment)
np.median(sentiment)
#pyplot.boxplot(sentiment)

0.004

In [210]:
keyword = []
pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
doc = nlp(example_text)
for token in doc:
    if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
        continue
    if(token.pos_ in pos_tag):
        keyword.append(token.text)

freq_word = Counter(keyword)
max_freq = Counter(keyword).most_common(1)[0][1]
for w in freq_word:
    freq_word[w] = (freq_word[w]/max_freq)
    
sent_strength={}
for sent in doc.sents:
    for word in sent:
        if word.text in freq_word.keys():
            if sent in sent_strength.keys():
                sent_strength[sent]+=freq_word[word.text]
            else:
                sent_strength[sent]=freq_word[word.text]
summary = []
limit =2
sorted_x = sorted(sent_strength.items(), key=lambda kv: kv[1], reverse=True)

counter = 0
for i in range(len(sorted_x)):
    summary.append(str(sorted_x[i][0]))

    counter += 1
    if(counter >= limit):
        break
        
' '.join(summary)

'„In den USA gibt es ein gewaltiges Potenzial für Batteriespeicher, und der Markt steht erst am Anfang“, sagt Philipp Schröder, Geschäftsführer Vertrieb und Marketing bei sonnen. Die Serienproduktion der sonnenBatterie für den US-Markt befindet sich in San José (Kalifornien) und sei aufgrund der hohen Nachfrage bereits ausgelastet, berichtet der Hersteller.'

In [208]:
a=[]
for token in doc:
  a.append(token.pos_)
Counter(a)

Counter({'ADJ': 19,
         'ADP': 29,
         'ADV': 17,
         'AUX': 7,
         'CCONJ': 13,
         'DET': 36,
         'NOUN': 58,
         'NUM': 3,
         'PART': 1,
         'PRON': 9,
         'PROPN': 13,
         'PUNCT': 32,
         'SCONJ': 1,
         'SPACE': 4,
         'VERB': 23,
         'X': 1})

In [0]:
Counter(doc.ents)

In [205]:
print(top_sentence(example_text, 5))

„In den USA gibt es ein gewaltiges Potenzial für Batteriespeicher, und der Markt steht erst am Anfang“, sagt Philipp Schröder, Geschäftsführer Vertrieb und Marketing bei sonnen. Die Serienproduktion der sonnenBatterie für den US-Markt befindet sich in San José (Kalifornien) und sei aufgrund der hohen Nachfrage bereits ausgelastet, berichtet der Hersteller.  sonnen GmbH (Wildpoldsried) ist ab sofort für Haushalte in den USA verfügbar. Aufgrund ihrer langen Lebensdauer von 10.000 Ladezyklen eignet sich die sonnenBatterie besonders für das tägliche Be- und Entladen, wie das beim Eigenverbrauch von Solarstrom typisch ist. Neben den Batteriemodulen sind auch der Wechselrichter, die Steuerung und die Messtechnik in einem Gehäuse enthalten und aufeinander abgestimmt.


In [0]:
url="https://www.derstandard.at/story/2000054503281/apple-arbeitet-an-eigenem-google-glass-konkurrenten"
headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 5.1.1; SM-G928X Build/LMY47X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.83 Mobile Safari/537.36'}
res=requests.get(url, headers=headers)
soup = BeautifulSoup(res.content, "html.parser")
article = soup.find("article", attrs={"class": "story-article"}).get_text()
#article = sent_tokenize(article)
article

'\n\n\n\nAugmented Reality\nApple arbeitet an eigenem Google-Glass-Konkurrenten\nMehrere hundert Entwickler arbeiten aktuell an AR-Lösungen – Neue iPhone-Features als erster Schritt\n\n\n\n\nGleich mehrfach hat Apple im vergangenen Jahr durchblicken lassen, dass man am Thema Augmented Reality interessiert ist, also der Anreicherung der Realität mit Computergrafiken. In einem aktuellen Artikel gibt Bloomberg nun erstmals Einblick in die konkreten Planungen, und dabei zeigt sich: Apple hat offenbar eine ganze Menge vor.\n\n\n\n'

In [0]:
url="https://www.pv-magazine.de/2019/08/22/der-solarboom-kommt-aktuelle-trends-sorgen-fuer-revolution-des-energiemarkts"
res=requests.get(url)
if res.status_code == 200:
  soup = BeautifulSoup(res.content, "html.parser")
  article = soup.find("div", attrs={"class": "content"})#.get_text()
  #article = sent_tokenize(article.replace("\n",""))
  soup.get_text()
else:
  print("Fehler: {}".format(res.status_code))

Fehler: 403


In [0]:
url="https://www.energynewsmagazine.at/de?query=##%C3%96sterreichs%20letztes%20kohlekraftwerk%20stellt%20betrieb%20ein"
res=requests.get(url)
if res.status_code == 200:
  soup = BeautifulSoup(res.content, "html.parser")
  article = soup.find("div", attrs={"class": "description"})#.get_text()
  #article = sent_tokenize(article.replace("\n",""))
  soup.get_text()
else:
  print("Fehler: {}".format(res.status_code))

SSLError: ignored